In [1]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline

""" Todo: 강의랑 달라진 점
Langchain과 HuggingFace의 버전 업데이트에 따라 더이상 Langchaing에서 제공하는 HuggingFaceHub 내부 의존성의 InterfaceAPI를 사용하지 않아. 
그래서 HuggingFaceEndpoint를 사용하도록 변경했다.

Langchaing과 HuggingFace-hub의 버전을 최신화하였다.
"""

load_dotenv(".env")

# prompt = PromptTemplate.from_template("[INST]What is te meaning of {word}? \n Please Answer to KOREAN.[/INST]")
# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#     model_kwargs={
#         "max_new_tokens": 250,
#     },
# )

# llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#     max_new_tokens=250,
# )

prompt = PromptTemplate.from_template("A {word} is a.")
llm = HuggingFacePipeline.from_model_id(
    model_id='gpt2',
    task='text-generation',
    pipeline_kwargs={
        "max_new_tokens": 50
    },
)

chain = prompt | llm

chain.invoke({
    "word": "tomato"
})

/Users/cyj/Documents/Projects/Python-Servers/Fullstack-GPT/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"A tomato is a. That's where it comes from. It grew on grapes for a few centuries, and over time they had to evolve, like most plants have.\n\nMolecular studies have been done on an entire race of tomato plants to detect genetic signatures of"

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.llms import GPT4All

""" Todo: 강의랑 달라진 점
GPT4ALL의 사용 방법이 강의랑 달라졌다. 이제 Window, Mac, Bin으로 어플리케이션을 받아야 Local 모델을 다운하든 할 수 있다. 

"""

load_dotenv(".env")

prompt = PromptTemplate.from_template("A {word} is a.")
llm = GPT4All(
    model="/Users/cyj/Library/Application Support/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf",
    callbacks=[],
    streaming=True
)

chain = prompt | llm

chain.invoke({
    "word": "tomato"
})

In [8]:
from langchain_community.chat_models import ChatOpenAI


def get_weather(lon, lat):
    print(f"call an api...")
    
function = {
    "name": "get_weather",
    "description": "function that takes longitude and latitude to find the weather of a place",
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {
                "type": "string",
                "description": "the longitude coordinate"
            },
            "lat": {
                "type": "string",
                "description": "the latitude coordinate"
            }
        },
    },
    "required": ["lon", "lat"],
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call="auto",
    functions=[
        function
    ]
)
prompt = PromptTemplate.from_template("Who is the weather in {city}")
chain = prompt | llm
response = chain.invoke({
    "city": "Seoul",
})

print(response)
print(response.additional_kwargs["function_call"]["arguments"])

import json
r = json.loads(response)
get_weather(r['lon'], r['lat'])

content='' additional_kwargs={'function_call': {'arguments': '{"lon":"126.9779","lat":"37.5665"}', 'name': 'get_weather'}} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 73, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-5aa6d516-1d5f-419c-9bce-2284d3efc1ea-0'
{"lon":"126.9779","lat":"37.5665"}


TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

print(response)


import json

for question in json.loads(response)["questions"]:
    print(question)

{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"500 AD","correct":false},{"answer":"100 BC","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by the ancient Romans?","answers":[{"answer":"Eiffel Tower","correct":false},{"answer":"Colosseum","correct":true},{"answer":"Big Ben","correct":false}]}]}
{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '500 AD', 'correct': False}, {'answer': '100 BC', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}]}
{'question': 'What famous structure in Rome was built by the ancient Roma